In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from sklearn.model_selection import train_test_split
from functools import *
import sys
import os
import json

PROJECT_ROOT = os.path.abspath(os.getcwd() + os.sep + os.pardir)
sys.path.insert(0, PROJECT_ROOT)

from lib.time_series_datasets import *
from lib.models import *
from lib.utility import *

DATA_ROOT  = os.path.join(PROJECT_ROOT, "data")
TUNER_ROOT = os.path.join(PROJECT_ROOT, "models", "tuner")
IMAGE_ROOT = os.path.join(PROJECT_ROOT, "images")
TB_ROOT    = os.path.join(os.path.abspath(os.sep), "tmp", "tensorboard")

BENCHMARKS_DIR = "benchmarks"
WEIGHTS_DIR = "weights"

print("       Data dir:", DATA_ROOT)
print("      Tuner dir:", TUNER_ROOT)
print("Tensorboard dir:", TB_ROOT)

SKIP = False

MAX_UNITS = 400
EPOCHS = 200
GUESSES = 10
PATIENCE = 10

#set the following values based on the specific dataset
OUTPUT_ACTIVATION = tf.keras.activations.softmax  # https://www.tensorflow.org/api_docs/python/tf/keras/activations  'softmax'
LOSS_FUNCTION = tf.keras.losses.SparseCategoricalCrossentropy()  # https://www.tensorflow.org/api_docs/python/tf/keras/losses  'sparse_categorical_crossentropy'

if not os.path.exists(TUNER_ROOT):
    os.makedirs(TUNER_ROOT)
if not os.path.exists(TB_ROOT):
    os.makedirs(TB_ROOT)

       Data dir: /dati/luca/Uni-Luca/Tesi/tesi/data
      Tuner dir: /dati/luca/Uni-Luca/Tesi/tesi/models/tuner
Tensorboard dir: /tmp/tensorboard



|        | ArticularyWordRecognition | CharacterTrajectories | Libras | SpokenArabicDigits |
|--------|:-------------------------:|:---------------------:|:------:|:------------------:|
| Input  |             9             |           3           |   2    |         13         |
| Output |            25             |          20           |   15   |         10         |

In [ ]:
DATASETS_NAMES = ["ArticularyWordRecognition", "CharacterTrajectories", "Libras", "SpokenArabicDigits"]
dataset_name = DATASETS_NAMES[2]

train_path  = os.path.join(DATA_ROOT, dataset_name, dataset_name + '_TRAIN.ts')
test_path = os.path.join(DATA_ROOT, dataset_name, dataset_name + '_TEST.ts')

x_train_all, y_train_all = load_sktime_dataset(train_path)  # Max shape serve per avere il test set e il train set della stessa lunghezza
x_test, y_test = load_sktime_dataset(test_path)

x_train, x_val, y_train, y_val = train_test_split(x_train_all, y_train_all, test_size=0.33, random_state=42, stratify=y_train_all)

train_set = (x_train.astype(np.float64), y_train.astype(np.float64))
val_set = (x_val.astype(np.float64), y_val.astype(np.float64))
test_set = (x_test.astype(np.float64), y_test.astype(np.float64))

features = x_train.shape[-1]
output_units = len(np.unique(y_test))  # Dataset must have one of each features

print("Input features: {}".format(features))
print("Output classes: {}".format(output_units))
print("Train shape:\n\tinput: {}\n\toutput: {}".format(x_train.shape, y_train.shape))
print("Validation shape:\n\tinput: {}\n\toutput: {}".format(x_val.shape, y_val.shape))
print("Test shape:\n\tinput: {}\n\toutput: {}".format(x_test.shape, y_test.shape))

Input features: 2
Output classes: 15
Train shape:
	input: (120, 45, 2)
	output: (120,)
Validation shape:
	input: (60, 45, 2)
	output: (60,)
Test shape:
	input: (180, 45, 2)
	output: (180,)


/dati/luca/Uni-Luca/Tesi/tesi/venv/lib/python3.9/site-packages/sktime/utils/data_io.py:63: FutureWarning: This function has moved to datasets/_data_io, this version will be removed in V0.10
  warn(


In [ ]:
def create_dir(root, _dataset_name):
    _path = os.path.join(root, _dataset_name)
    if not os.path.exists(_path):
        os.makedirs(_path)
    return _path

tuner_path = create_dir(TUNER_ROOT, dataset_name)
image_path = create_dir(IMAGE_ROOT, dataset_name)
benchmarks_path = create_dir(image_path, BENCHMARKS_DIR)
json_path = os.path.join(benchmarks_path, "benchmarks.json")
weight_path = create_dir(image_path, WEIGHTS_DIR)

In [ ]:
benchmarks = []
if os.path.exists(json_path):
    with open(json_path, 'r') as jsonfile:
        tmp = json.load(jsonfile)
        for values in tmp:
            benchmarks.append(Benchmark.fromJson(values))
else:
    with open(json_path, 'w') as jsonfile:
        json.dump([], jsonfile)

In [ ]:
def build_best_ESN1(hp):
    tmp_model = ESN1(units=hp.Int('units', min_value=50, max_value=MAX_UNITS),
                     connectivity=hp.Float('connectivity 0',  min_value=0.0, max_value=1.),
                     output_units=output_units,
                     output_activation=OUTPUT_ACTIVATION,
                     input_scaling=hp.Float('input scaling', min_value=0.1, max_value=1.5, step=0.1),
                     bias_scaling=hp.Float('bias scaling', min_value=0.1, max_value=1.5, step=0.1),
                     spectral_radius=hp.Float('spectral radius', min_value=0.01, max_value=2., sampling='log'),
                     leaky=hp.Float('leaky', min_value=0.1, max_value=1, step=0.1))

    alpha = hp.Float('learning rate', min_value=1e-5, max_value=1e-1, sampling='log')
    tmp_model.compile(
        optimizer=keras.optimizers.RMSprop(alpha),
        loss=LOSS_FUNCTION,
        metrics=['accuracy'])
    return tmp_model

def build_best_ESN2(hp):
    global_sr = hp.Boolean('global spectral radius')
    if global_sr:
        sr = hp.Float('spectral radius', min_value=0.01, max_value=2., sampling='log', parent_name='global spectral radius', parent_values=True)
    else:
        sr = [hp.Float('spectral radius ' + str(i), min_value=0.01, max_value=2., sampling='log', parent_name='global spectral radius', parent_values=False)
              for i in range(features)]

    connectivity = [hp.Float('connectivity ' + str(i), min_value=0.0, max_value=1.) for i in range(features)]

    tmp_model = ESN2(units=hp.Int('units', min_value=50, max_value=MAX_UNITS),
                     sub_reservoirs=features,
                     output_units=output_units,
                     connectivity=connectivity,
                     output_activation=OUTPUT_ACTIVATION,
                     input_scaling=hp.Float('input scaling', min_value=0.1, max_value=1.5, step=0.1),
                     bias_scaling=hp.Float('bias scaling', min_value=0.1, max_value=1.5, step=0.1),
                     spectral_radius=sr,
                     leaky=hp.Float('leaky', min_value=0.1, max_value=1, step=0.1),
                     global_sr=global_sr)

    alpha = hp.Float('learning rate', min_value=1e-5, max_value=1e-1, sampling='log')
    tmp_model.compile(
        optimizer=keras.optimizers.RMSprop(alpha),
        loss=LOSS_FUNCTION,
        metrics=['accuracy'])
    return tmp_model

def build_best_ESN3(hp):
    global_sr = hp.Boolean('global spectral radius')
    if global_sr:
        sr = hp.Float('spectral radius', min_value=0.01, max_value=2., sampling='log', parent_name='global spectral radius', parent_values=True)
    else:
        sr = [hp.Float('spectral radius ' + str(i), min_value=0.01, max_value=2., sampling='log', parent_name='global spectral radius', parent_values=False)
              for i in range(features)]

    connectivity = [[hp.Float('connectivity ' + str(i), min_value=0.0, max_value=1.) if i == j else
                     hp.Float('connectivity ' + str(i) + '->' + str(j), min_value=0.0, max_value=1.)
                     for i in range(features)]
                     for j in range(features)]

    tmp_model = ESN3(units=hp.Int('units', min_value=50, max_value=MAX_UNITS),
                     sub_reservoirs=features,
                     connectivity=connectivity,
                     output_units=output_units,
                     output_activation=OUTPUT_ACTIVATION,
                     input_scaling=hp.Float('input scaling', min_value=0.1, max_value=1.5, step=0.1),
                     bias_scaling=hp.Float('bias scaling', min_value=0.1, max_value=1.5, step=0.1),
                     spectral_radius=sr,
                     leaky=hp.Float('leaky', min_value=0.1, max_value=1., step=0.1),
                     global_sr=global_sr)

    alpha = hp.Float('learning rate', min_value=1e-5, max_value=1e-1, sampling='log')
    tmp_model.compile(
        optimizer=keras.optimizers.RMSprop(alpha),
        loss=LOSS_FUNCTION,
        metrics=['accuracy'])
    return tmp_model

def build_best_ESN4(hp):
    global_sr = hp.Boolean('global spectral radius')
    if global_sr:
        sr = hp.Float('spectral radius', min_value=0.01, max_value=2., sampling='log', parent_name='global spectral radius', parent_values=True)
    else:
        sr = [hp.Float('spectral radius ' + str(i), min_value=0.01, max_value=2., sampling='log', parent_name='global spectral radius', parent_values=False)
              for i in range(features)]

    connectivity = [[hp.Float('connectivity ' + str(i), min_value=0.0, max_value=1.) if i == j else
                     hp.Float('connectivity ' + str(i) + '->' + str(j), min_value=0.0, max_value=1)
                     for i in range(features)]
                     for j in range(features)]

    partitions = [hp.Float('partition ' + str(i), min_value=0., max_value=1.0) for i in range(features)]
    total = sum(partitions)
    partitions = list(map(lambda _x: 0 if total == 0 else _x / total, partitions))  # Normalize the partition vector now sum(partitions) == 1.

    tmp_model = ESN4(units=hp.Int('units', min_value=50, max_value=MAX_UNITS),
                     sub_reservoirs=features,
                     partitions=partitions,
                     connectivity=connectivity,
                     output_units=output_units,
                     output_activation=OUTPUT_ACTIVATION,
                     input_scaling=hp.Float('input scaling', min_value=0.1, max_value=1.5, step=0.1),
                     bias_scaling=hp.Float('bias scaling', min_value=0.1, max_value=1.5, step=0.1),
                     spectral_radius=sr,
                     leaky=hp.Float('leaky', min_value=0.1, max_value=1, step=0.1),
                     global_sr=global_sr)

    alpha = hp.Float('learning rate', min_value=1e-5, max_value=1e-1, sampling='log')
    tmp_model.compile(
        optimizer=keras.optimizers.RMSprop(alpha),
        loss=LOSS_FUNCTION,
        metrics=['accuracy'])
    return tmp_model

best_models = {
    #'ESN1': build_best_ESN1,
    'ESN2': build_best_ESN2,
    #'ESN3': build_best_ESN3,
    #'ESN4': build_best_ESN4,
}

### Find best models

In [ ]:
%time
exp_name = "Best"

for model_name, build_fn in best_models.items():
    if is_benchmarked(benchmarks, model_name, exp_name) and SKIP:
            continue
    tf.random.set_seed(42)
    model = tune_and_test(model_name, build_fn, exp_name, train_set, val_set, test_set,
                          EPOCHS, PATIENCE, GUESSES,
                          benchmarks, tuner_path=tuner_path)

    model.plot(model_name, exp_name, path=weight_path, show=False)
send_notification("Best is done", "All model done")

Trial 254 Complete [00h 00m 01s]
val_accuracy: 0.2666666805744171

Best val_accuracy So Far: 0.5166666507720947
Total elapsed time: 00h 03m 19s
INFO:tensorflow:Oracle triggered exit
Start 10 benchmarks:
6/6 [==============================] - 0s 587us/step - loss: 1.6008 - accuracy: 0.4667
Not already present in benchmarks. Adding...
     Model : ESN2
Experiment : Best
  Accuracy : 36.61±9.47 %
      Loss : 1.94±0.28
Build time : 2.71±0.47s



In [ ]:
for b in benchmarks:
    print(b.model, b.experiment, b.timestamp)

ESN1 best 1641151163
ESN2 best 1641151447
ESN3 best 1641151725
ESN4 best 1641151991
ESN1 Units 50 Connectivity 1 1641152227
ESN2 Units 50 Connectivity 1 1641152467
ESN3 Units 50 Connectivity 1 1641152715
ESN4 Units 50 Connectivity 1 1641152973
ESN1 Units 75 Connectivity 1 1641153222
ESN2 Units 75 Connectivity 1 1641153495
ESN3 Units 75 Connectivity 1 1641153755
ESN4 Units 75 Connectivity 1 1641154006
ESN1 Units 112 Connectivity 1 1641154249
ESN2 Units 112 Connectivity 1 1641154502
ESN3 Units 112 Connectivity 1 1641154754
ESN4 Units 112 Connectivity 1 1641155010
ESN1 Units 168 Connectivity 1 1641155283
ESN2 Units 168 Connectivity 1 1641155556
ESN3 Units 168 Connectivity 1 1641155822
ESN4 Units 168 Connectivity 1 1641156076
ESN1 Units 253 Connectivity 1 1641156329
ESN2 Units 253 Connectivity 1 1641156608
ESN3 Units 253 Connectivity 1 1641156894
ESN4 Units 253 Connectivity 1 1641157176
ESN1 Units 379 Connectivity 1 1641157465
ESN2 Units 379 Connectivity 1 1641157801
ESN3 Units 379 Connect

## Find best hyperparameters for each model

In [ ]:
def build_ESN1(units, _, hp):
    units = hp.Fixed('units', units)
    hp.Fixed('global spectral radius', True)

    tmp_model = ESN1(units=units,
                     connectivity=hp.Fixed('connectivity 0', 1.),
                     output_units=output_units,
                     output_activation=OUTPUT_ACTIVATION,
                     input_scaling=hp.Float('input scaling', min_value=0.1, max_value=1.5, step=0.1),
                     bias_scaling=hp.Float('bias scaling', min_value=0.1, max_value=1.5, step=0.1),
                     spectral_radius=hp.Float('spectral radius', min_value=0.1, max_value=1.5, step=0.1),
                     leaky=hp.Float('leaky', min_value=0.1, max_value=1, step=0.1))

    alpha = hp.Float('learning rate', min_value=1e-5, max_value=1e-1, sampling='log')
    tmp_model.compile(
        optimizer=keras.optimizers.RMSprop(alpha),
        loss=LOSS_FUNCTION,
        metrics=['accuracy'])
    return tmp_model


def build_ESN2(units, global_sr, hp):
    units = hp.Fixed('units', units)
    global_sr = hp.Boolean('global spectral radius')
    connectivity = [hp.Fixed('connectivity ' + str(i), 1.) for i in range(features)]

    tmp_model = ESN2(units=units,
                     sub_reservoirs=features,
                     connectivity=connectivity,
                     output_units=output_units,
                     output_activation=OUTPUT_ACTIVATION,
                     input_scaling=hp.Float('input scaling', min_value=0.1, max_value=1.5, step=0.1),
                     bias_scaling=hp.Float('bias scaling', min_value=0.1, max_value=1.5, step=0.1),
                     spectral_radius=hp.Float('spectral radius', min_value=0.1, max_value=1.5, step=0.1),
                     leaky=hp.Float('leaky', min_value=0.1, max_value=1, step=0.1),
                     global_sr=global_sr)

    alpha = hp.Float('learning rate', min_value=1e-5, max_value=1e-1, sampling='log')
    tmp_model.compile(
        optimizer=keras.optimizers.RMSprop(alpha),
        loss=LOSS_FUNCTION,
        metrics=['accuracy'])
    return tmp_model


def build_ESN3(units, global_sr, hp):
    units = hp.Fixed('units', units)
    global_sr = hp.Boolean('global spectral radius')

    connectivity = [[hp.Fixed('connectivity ' + str(i), 1.) if i == j else
                     hp.Float('connectivity ' + str(i) + '->' + str(j), min_value=0.0, max_value=1.)
                     for i in range(features)]
                     for j in range(features)]

    tmp_model = ESN3(units=units,
                     sub_reservoirs=features,
                     connectivity=connectivity,
                     output_units=output_units,
                     output_activation=OUTPUT_ACTIVATION,
                     input_scaling=hp.Float('input_scaling', min_value=0.1, max_value=1.5, step=0.1),
                     bias_scaling=hp.Float('bias_scaling', min_value=0.1, max_value=1.5, step=0.1),
                     spectral_radius=hp.Float('spectral_radius', min_value=0.1, max_value=1.5, step=0.1),
                     leaky=hp.Float('leaky', min_value=0.1, max_value=1, step=0.1),
                     global_sr=global_sr)

    alpha = hp.Float('learning_rate', min_value=1e-5, max_value=1e-1, sampling='log')
    tmp_model.compile(
        optimizer=keras.optimizers.RMSprop(alpha),
        loss=LOSS_FUNCTION,
        metrics=['accuracy'])
    return tmp_model


def build_ESN4(units, global_sr, hp):
    units = hp.Fixed('units', units)
    connectivity = [[hp.Fixed('connectivity ' + str(i), 1.) if i == j else
                     hp.Float('connectivity ' + str(i) + '->' + str(j), min_value=0.0, max_value=1)
                    for i in range(features)]
                   for j in range(features)]

    partitions = [hp.Float('partition ' + str(i), min_value=0., max_value=1.0) for i in range(features)]
    total = sum(partitions)
    partitions = list(map(lambda _x: 0 if total == 0 else _x / total, partitions))  # Normalize the partition vector now sum(partitions) == 1.

    tmp_model = ESN4(units=units,
                     sub_reservoirs=features,
                     partitions=partitions,
                     connectivity=connectivity,
                     output_units=output_units,
                     output_activation=OUTPUT_ACTIVATION,
                     input_scaling=hp.Float('input scaling', min_value=0.1, max_value=1.5, step=0.1),
                     bias_scaling=hp.Float('bias scaling', min_value=0.1, max_value=1.5, step=0.1),
                     spectral_radius=hp.Float('spectral radius', min_value=0.1, max_value=1.5, step=0.1),
                     leaky=hp.Float('leaky', min_value=0.1, max_value=1, step=0.1),
                     global_sr=hp.Boolean('global spectral radius'))

    alpha = hp.Float('learning rate', min_value=1e-5, max_value=1e-1, sampling='log')
    tmp_model.compile(
        optimizer=keras.optimizers.RMSprop(alpha),
        loss=LOSS_FUNCTION,
        metrics=['accuracy'])
    return tmp_model

In [ ]:
%time

#  Units follow this rule units(x) = (1.5 ** x) * 50  for x in [0; 5]
experiments = {
    'Units 50 Connectivity 1': (50, None),
    'Units 75 Connectivity 1': (75, None),
    'Units 112 Connectivity 1': (122, None),
    'Units 168 Connectivity 1': (168, None),
    'Units 253 Connectivity 1': (253, None),
    'Units 379 Connectivity 1': (379, None),
}

models = {
    'ESN1': build_ESN1,
    'ESN2': build_ESN2,
    'ESN3': build_ESN3,
    'ESN4': build_ESN4,
}

for exp_name, fn_params in experiments.items():
    for model_name, build_fn in models.items():
        if is_benchmarked(benchmarks, model_name, exp_name) and SKIP:
            continue
        tf.random.set_seed(42)
        build_fn = partial(build_fn, *fn_params)
        model = tune_and_test(model_name, build_fn, exp_name, train_set, val_set, test_set, EPOCHS, PATIENCE, GUESSES, benchmarks, tuner_path=tuner_path)
        model.plot(model_name, exp_name, path=weight_path)

send_notification("Done", "All model done")

CPU times: user 2 µs, sys: 0 ns, total: 2 µs
Wall time: 4.77 µs
INFO:tensorflow:Reloading Oracle from existing project /dati/luca/Uni-Luca/Tesi/tesi/models/tuner/Libras/ESN1 Units 50 Connectivity 1 hyperband/oracle.json
INFO:tensorflow:Reloading Tuner from /dati/luca/Uni-Luca/Tesi/tesi/models/tuner/Libras/ESN1 Units 50 Connectivity 1 hyperband/tuner0.json
INFO:tensorflow:Oracle triggered exit
Start 10 benchmarks:
6/6 [==============================] - 0s 956us/step - loss: 2.4176 - accuracy: 0.2778
Already present. Overwriting..
     Model : ESN1
Experiment : Units 50 Connectivity 1
  Accuracy : 30.89±8.80 %
      Loss : 2.27±0.20
Build time : 0.81±0.19s

INFO:tensorflow:Reloading Oracle from existing project /dati/luca/Uni-Luca/Tesi/tesi/models/tuner/Libras/ESN2 Units 50 Connectivity 1 hyperband/oracle.json


/dati/luca/Uni-Luca/Tesi/tesi/lib/models.py:127: UserWarning: Matplotlib is currently using module://matplotlib_inline.backend_inline, which is a non-GUI backend, so cannot show the figure.
  fig.show()


INFO:tensorflow:Reloading Tuner from /dati/luca/Uni-Luca/Tesi/tesi/models/tuner/Libras/ESN2 Units 50 Connectivity 1 hyperband/tuner0.json
INFO:tensorflow:Oracle triggered exit
Start 10 benchmarks:
6/6 [==============================] - 0s 446us/step - loss: 2.4678 - accuracy: 0.2000
Already present. Overwriting..
     Model : ESN2
Experiment : Units 50 Connectivity 1
  Accuracy : 30.67±4.75 %
      Loss : 2.15±0.16
Build time : 2.66±0.62s

INFO:tensorflow:Reloading Oracle from existing project /dati/luca/Uni-Luca/Tesi/tesi/models/tuner/Libras/ESN3 Units 50 Connectivity 1 hyperband/oracle.json
INFO:tensorflow:Reloading Tuner from /dati/luca/Uni-Luca/Tesi/tesi/models/tuner/Libras/ESN3 Units 50 Connectivity 1 hyperband/tuner0.json
INFO:tensorflow:Oracle triggered exit
Start 10 benchmarks:
6/6 [==============================] - 0s 902us/step - loss: 1.7856 - accuracy: 0.4333
Already present. Overwriting..
     Model : ESN3
Experiment : Units 50 Connectivity 1
  Accuracy : 38.61±8.36 %
    

KeyboardInterrupt: 

In [ ]:
class BenchmarkEncoder(json.JSONEncoder):
    def default(self, obj):
        if isinstance(obj, lib.utility.Benchmark):
            return obj.toJson()
        return json.JSONEncoder.default(self, obj)

with open(json_path, "w") as out_file:
    json.dump(benchmarks, out_file, cls=BenchmarkEncoder, indent=4)